In [16]:
import botometer
import pandas as pd

In [7]:
mashape_key = '5e62e89027msh4b725e4348c6340p1edfd8jsn9f29c99b07e9'

twitter_app_auth = {
    'consumer_key': 'tctr9T5rClcru9HR6l7Y5kAdP',
    'consumer_secret': 'qsJcfVkrc5IJElL6KQ6aeBPuEBvhJJlx00zefv1ppVZEjQv6fk',
    'access_token': '39675338-Hlg4DdfCReGPXzMAbLkBnWjb45hm0Za4aMN6IcpV1',
    'access_token_secret': 'fRtKq262hAS1MXvpacN2aX7Dh6WWirJFTCB6kX1USSw9C',
  }


bom = botometer.Botometer(wait_on_ratelimit=True,
                          mashape_key=mashape_key,
                          **twitter_app_auth)


In [17]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("/Users/andresgf91/Capstone/Data/SQL_META_HARVEY.sqlite")
all_ids = pd.read_sql_query("select user_id_str from all_texas_tweets;", conn)
all_ids = set(all_ids['user_id_str'])

print('finished getting business from  SQL')

all_ids = set(all_ids)
len(all_ids)

finished getting business from  SQL


21085

In [46]:
all_ids = [int(i) for i in all_ids]

prev_results = pd.read_csv(filepath_or_buffer="/Users/andresgf91/Capstone/Data/botometer_2.csv")
prev_results.head(10)
prev_ids = set(prev_results['ids'])

In [47]:
still_needs_searching = [i for i in all_ids if i not in prev_ids]
print(len(still_needs_searching))

892


In [49]:
to_search = still_needs_searching[401:891]

In [50]:
ls = list()
accounts = to_search
for screen_name, result in bom.check_accounts_in(accounts):
    ls.append(result)
    
print('Finished running botometer scores...')
len(ls)

Finished running botometer scores...


490

In [51]:
import pandas as pd

old_botometer_df = pd.read_csv(filepath_or_buffer="~/Capstone/Data/botometer_2.csv")

try:
    del old_botometer_df['Unnamed: 0']
except KeyError: 
    pass

errors = list()

for i in range(0,len(ls)):
    try:
        d=ls[i]['user']
        d=ls[i]['scores']
    except KeyError:
        errors.append(i)
        
ls = [ls[i] for i in range(0,len(ls)) if i not in errors]

ids = list(ls[i]['user']['id_str'] for i in range(0,len(ls)))
english_score = list(ls[i]['scores']['english'] for i in range(0,len(ls)))
universal_score = list(ls[i]['scores']['universal'] for i in range(0,len(ls)))

new_botometer_df = pd.DataFrame({'ids':ids,'english_score':english_score,'universal_score':universal_score})

botometer_df = new_botometer_df.append(old_botometer_df, ignore_index = True, sort=False)
botometer_df = pd.DataFrame.drop_duplicates(botometer_df,subset='ids')

pd.DataFrame.to_csv(botometer_df,path_or_buf="~/Capstone/Data/botometer_2.csv", index= False)

len(botometer_df)

20652